In [1]:
import os
from utils.auxiliary_func import *
from utils.plots_func import *
import config.const as const

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import ast
import json

## Define Parameters

In [27]:
DATA_PREPARATION_DIR = const.DATA_PREPARATION_DIR

## Create data frame of GNN results

In [3]:
tissue_resolution = 'patch_tissue'
# Extract features
max_dist = 30
patch_size = 1500
overlap = 0.75
feature_type = 'lifetime'

In [4]:
model_params_dict = params.model_params[1]
# model_params_dict['epochs'] = 50
model_params_dict

{'model_type': 'GAT',
 'hidden_layers': [64, 128, 64],
 'output_size': 1,
 'pooling': 'max',
 'heads': 1,
 'weight_decay': 0.0001,
 'epochs': 50,
 'class_ratio': 1.0,
 'dropout_rate': 0.1,
 'batch_size': 32,
 'test_ratio': 0.1,
 'lr': 0.0001}

### GNN - original lifetime values

> **Note on `date` parameter:**  
> The variable `date = '21_06_2025'` refers to the **timestamped directory** where model results are saved.  
> Make sure to **update this date** to match the folder name corresponding to your specific model training run.  

In [41]:
date = '21_06_2025'

In [42]:
GNN_DIR=const.GNN_DIR

In [43]:
model_dir = build_gnn_model_dir_name(model_params_dict)

gnn_results_dir = os.path.join(
    # const.GNN_DIR,
    GNN_DIR,
    feature_type,
    tissue_resolution,
    f"max_distance_{max_dist}",
    "pytorch_geo",
    f"size_{patch_size}_overlap_{overlap}",
    "results",
    date)

model_result_dir = os.path.join(gnn_results_dir, model_dir)

print(model_result_dir)

/sise/assafzar-group/assafzar/reut/FLIM/gnn/lifetime/patch_tissue/max_distance_30/pytorch_geo/size_1500_overlap_0.75/results/21_06_2025/batch_size_32_class_ratio_1p0_dropout_rate_0p1_epochs_50_heads_1_hidden_layers_[64, 128, 64]_lr_0p0001_model_type_GAT_output_size_1_pooling_max_test_ratio_0p1_weight_decay_0p0001


In [44]:
# Read the DataFrame
seeds_gnn_file_path = os.path.join(model_result_dir, "seed_results.pkl")
seeds_gnn_df = pd.read_pickle(seeds_gnn_file_path)
seeds_gnn_df.head(2)

,seed_val,auc_score,fpr,tpr,auc_agg,fpr_agg,tpr_agg,auc_agg_filter,fpr_agg_filter,tpr_agg_filter
0,1825,0.623534,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0001231830500123183, 0.001231830500123...",0.698551,"[0.0, 0.043478260869565216, 0.0434782608695652...","[0.0, 0.0, 0.1, 0.1, 0.13333333333333333, 0.13...",0.686957,"[0.0, 0.043478260869565216, 0.0434782608695652...","[0.0, 0.0, 0.1, 0.1, 0.13333333333333333, 0.13..."
1,410,0.706783,"[0.0, 0.00017316017316017316, 0.00069264069264...","[0.0, 0.0, 0.0, 0.0002463661000246366, 0.00098...",0.742029,"[0.0, 0.0, 0.043478260869565216, 0.04347826086...","[0.0, 0.03333333333333333, 0.03333333333333333...",0.715942,"[0.0, 0.0, 0.043478260869565216, 0.04347826086...","[0.0, 0.03333333333333333, 0.03333333333333333..."


In [45]:
seed_list_gnn, auc_list_gnn, fpr_list_gnn, tpr_list_gnn = extract_gnn_seed_results(model_result_dir,  'auc_agg', 'tpr_agg', 'fpr_agg')

data = {
    'seed_val': seed_list_gnn,
    'auc_agg': auc_list_gnn,
    'fpr_agg': [json.dumps(f.tolist()) for f in fpr_list_gnn],
    'tpr_agg': [json.dumps(t.tolist()) for t in tpr_list_gnn],
}
gnn_results_df = pd.DataFrame(data)

csv_save_path = os.path.join(DATA_PREPARATION_DIR, f"patch_{patch_size}_gnn_model_agg_results_patch_wise.csv")
gnn_results_df.to_csv(csv_save_path, index=False)
print(f"Saved selected columns to: {csv_save_path}")

Saved selected columns to: /sise/assafzar-group/assafzar/reut/Test_FLIM/from_FLIM_dir/figure_results/data_preparation/patch_1500_gnn_model_agg_results_patch_wise.csv


### GNN shuffling lifetime values

In [46]:
model_dir = build_gnn_model_dir_name(model_params_dict)

gnn_shuffling_results_path = os.path.join(
    # const.GNN_DIR,
    GNN_DIR,
    feature_type,
    tissue_resolution,
    f"max_distance_{max_dist}",
    "pytorch_geo",
    'shuffling_lifetime',
    f"size_{patch_size}_overlap_{overlap}",
    "results")

gnn_shuffling_results_dir = os.path.join(gnn_shuffling_results_path, model_dir)

print(gnn_shuffling_results_dir)

/sise/assafzar-group/assafzar/reut/FLIM/gnn/lifetime/patch_tissue/max_distance_30/pytorch_geo/shuffling_lifetime/size_1500_overlap_0.75/results/batch_size_32_class_ratio_1p0_dropout_rate_0p1_epochs_50_heads_1_hidden_layers_[64, 128, 64]_lr_0p0001_model_type_GAT_output_size_1_pooling_max_test_ratio_0p1_weight_decay_0p0001


In [47]:
seeds_gnn_shuffling_file_path = os.path.join(gnn_shuffling_results_dir, "seed_results.pkl")
seeds_gnn_shuffling_df = pd.read_pickle(seeds_gnn_shuffling_file_path)
seeds_gnn_shuffling_df.head(2)

,seed_val,auc_score,fpr,tpr,auc_agg,fpr_agg,tpr_agg,auc_agg_filter,fpr_agg_filter,tpr_agg_filter
0,489,0.635203,"[0.0, 0.0, 0.0, 0.0003463203463203463, 0.00034...","[0.0, 0.0001231830500123183, 0.000492732200049...",0.684058,"[0.0, 0.0, 0.043478260869565216, 0.04347826086...","[0.0, 0.03333333333333333, 0.03333333333333333...",0.624638,"[0.0, 0.0, 0.043478260869565216, 0.04347826086...","[0.0, 0.03333333333333333, 0.03333333333333333..."
1,410,0.683414,"[0.0, 0.0, 0.0, 0.0003463203463203463, 0.00034...","[0.0, 0.0001231830500123183, 0.000369549150036...",0.720290,"[0.0, 0.043478260869565216, 0.0434782608695652...","[0.0, 0.0, 0.2, 0.2, 0.23333333333333334, 0.23...",0.710145,"[0.0, 0.043478260869565216, 0.0434782608695652...","[0.0, 0.0, 0.13333333333333333, 0.133333333333..."


In [48]:
seed_list_gnn_shuffling, auc_list_gnn_shuffling, fpr_list_gnn_shuffling, tpr_list_gnn_shuffling = extract_gnn_seed_results(gnn_shuffling_results_dir,  'auc_agg', 'tpr_agg', 'fpr_agg')
data = {
    'seed_val': seed_list_gnn_shuffling,
    'auc_agg': auc_list_gnn_shuffling,
    'fpr_agg': [json.dumps(f.tolist()) for f in fpr_list_gnn_shuffling],
    'tpr_agg': [json.dumps(t.tolist()) for t in tpr_list_gnn_shuffling],
}
gnn_shuffling_results_df = pd.DataFrame(data)

# csv_save_path = os.path.join(DATA_PREPARATION_DIR, f"patch_{patch_size}_gnn_shuffling_model_agg_results_patch_wise.csv")
# gnn_shuffling_results_df.to_csv(csv_save_path, index=False)
# print(f"Saved selected columns to: {csv_save_path}")

### GNN structure results

In [49]:
model_dir = build_gnn_model_dir_name(model_params_dict)

gnn_structure_results_path = os.path.join(
    # const.GNN_DIR,
    GNN_DIR,
    feature_type,
    tissue_resolution,
    f"max_distance_{max_dist}",
    "pytorch_geo",
    'structure_only',
    f"size_{patch_size}_overlap_{overlap}",
    "results")

gnn_structure_directory_path = os.path.join(gnn_structure_results_path, model_dir)
print(gnn_structure_directory_path)

/sise/assafzar-group/assafzar/reut/FLIM/gnn/lifetime/patch_tissue/max_distance_30/pytorch_geo/structure_only/size_1500_overlap_0.75/results/batch_size_32_class_ratio_1p0_dropout_rate_0p1_epochs_50_heads_1_hidden_layers_[64, 128, 64]_lr_0p0001_model_type_GAT_output_size_1_pooling_max_test_ratio_0p1_weight_decay_0p0001


In [50]:
seeds_gnn_structure_file_path = os.path.join(gnn_structure_directory_path, "seed_results.pkl")
seeds_gnn_structure_df = pd.read_pickle(seeds_gnn_structure_file_path)
seeds_gnn_structure_df.head(2)

,seed_val,auc_score,fpr,tpr,auc_agg,fpr_agg,tpr_agg,auc_agg_filter,fpr_agg_filter,tpr_agg_filter
0,435,0.406333,"[0.0, 0.3047619047619048, 0.5111688311688312, ...","[0.0, 0.19081054446908105, 0.3808819906380882,...",0.489855,"[0.0, 0.0, 0.17391304347826086, 0.260869565217...","[0.0, 0.03333333333333333, 0.13333333333333333...",0.489855,"[0.0, 0.0, 0.17391304347826086, 0.260869565217...","[0.0, 0.03333333333333333, 0.13333333333333333..."
1,489,0.384893,"[0.0, 0.3047619047619048, 0.5298701298701298, ...","[0.0, 0.19081054446908105, 0.36659275683665926...",0.471014,"[0.0, 0.043478260869565216, 0.1739130434782608...","[0.0, 0.0, 0.16666666666666666, 0.2, 0.4, 0.5,...",0.471014,"[0.0, 0.043478260869565216, 0.1739130434782608...","[0.0, 0.0, 0.16666666666666666, 0.2, 0.4, 0.5,..."


In [51]:
seed_list_gnn_structure, auc_list_gnn_structure, fpr_list_gnn_structure, tpr_list_gnn_structure = extract_gnn_seed_results(gnn_structure_directory_path,  'auc_agg', 'tpr_agg', 'fpr_agg')
data = {
    'seed_val': seed_list_gnn_structure,
    'auc_agg': auc_list_gnn_structure,
    'fpr_agg': [json.dumps(f.tolist()) for f in fpr_list_gnn_structure],
    'tpr_agg': [json.dumps(t.tolist()) for t in tpr_list_gnn_structure],
}
gnn_structure_results_df = pd.DataFrame(data)

# csv_save_path = os.path.join(DATA_PREPARATION_DIR, f"patch_{patch_size}_gnn_structure_model_agg_results_patch_wise.csv")
# gnn_structure_results_df.to_csv(csv_save_path, index=False)
# print(f"Saved selected columns to: {csv_save_path}")

### Create one data frame of AUC score for each GNN model 

In [52]:
AUC_scores = list(auc_list_gnn) + list(auc_list_gnn_shuffling) + list(auc_list_gnn_structure)
Model = (['GNN'] * len(auc_list_gnn) +
         ['GNN_shuffling'] * len(auc_list_gnn_shuffling) +
         ['GNN_structure'] * len(auc_list_gnn_structure))

auc_df = pd.DataFrame({'AUC': AUC_scores, 'Model': Model})
auc_df.head()


,AUC,Model
0,0.698551,GNN
1,0.742029,GNN
2,0.730435,GNN
3,0.679710,GNN
4,0.720290,GNN


In [53]:
output_auc_models_path = os.path.join(DATA_PREPARATION_DIR, 'auc_df_gnn_models.csv')
auc_df.to_csv(output_auc_models_path, index=False)